# **Medida de similaridade entre textos**






#### **Introdução**

---

A medida de similaridade entre textos é uma técnica utilizada no processamento de linguagem natural (NLP) para comparar a similaridade ou proximidade entre dois ou mais textos. Essa técnica é muito útil em diversas aplicações, por exemplo:

- Recuperação de informações
- Análise de sentimentos
- Classificação de textos
- Detecção de plágio
- Resumo automático

<img src="https://media.giphy.com/media/dQpUkK59l5Imxsh8jN/giphy.gif" alt="win" width="600"/>

A medida de similaridade entre textos pode ser realizada de diversas maneiras, como por exemplo, utilizando medidas baseadas em frequência de termos, medidas baseadas em modelos de tópicos, medidas baseadas em redes semânticas e medidas baseadas em aprendizado de máquina. A escolha da medida mais adequada depende do tipo de problema e do tipo de texto que está sendo analisado.

#### **Objetivo**

---

Neste projeto o objetivo é devolver um score de semelhança entre frases

#### **Sobre os dados**

---

Dada a seguinte frase original:

*Olhando para a escala na parede, qual valor indicaria melhor a sua dor hoje?*

Faremos uma comparação com as 3 frases a seguir, devolvendo o score de semelhança entre a original e a comparativa:

- *De acordo com a escala de dor ali na parede, qual valor você acha que mais representa a sua dor?*
- *De 0 a 10, qual o nível de intensidade da sua dor atualmente?*
- *Qual a intensidade da sua dor?*

#### **Primeiros passos**

---

Vamos começar criando uma lista com a frase original e as frases comparativas:

In [1]:
# Frase original 

x = 'Olhando para a escala na parede, qual valor indicaria melhor a sua dor hoje?'

In [2]:
# Frases comparativas

y = ['De acordo com a escala de dor ali na parede, qual valor você acha que mais representa a sua dor?',
     'De 0 a 10, qual o nível de intensidade da sua dor atualmente?',
     'Qual a intensidade da sua dor?']

Criado nossa lista, vamos concatenar nossas frases na variável 'documents':

In [3]:
documents = [x] + y
documents

['Olhando para a escala na parede, qual valor indicaria melhor a sua dor hoje?',
 'De acordo com a escala de dor ali na parede, qual valor você acha que mais representa a sua dor?',
 'De 0 a 10, qual o nível de intensidade da sua dor atualmente?',
 'Qual a intensidade da sua dor?']

#### **Limpeza e tratamento**

---

Nossas frases estão com pontuações e caracteres especiais, para melhor desempenho deste estudo vamos realizar uma limpeza e aplicar em nosso conjunto de dados:

In [ ]:
# Importando libs necessárias para realizar a limpeza

import string
!pip install unidecode
from unidecode import unidecode
import re

Realizando limpeza e adicionando as frases na variável "texto_limpo":

In [5]:
def clean_text(text):
    # Remove as pontuações
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove os acentos e deixa tudo minúsculo
    text = unidecode(text).lower()

    # Remove os números
    text = re.sub(r'\d+', '', text)

    return text

# Limpa as frases
texto_limpo = [clean_text(frase) for frase in documents]

In [6]:
# Visualização das frases limpas

texto_limpo

['olhando para a escala na parede qual valor indicaria melhor a sua dor hoje',
 'de acordo com a escala de dor ali na parede qual valor voce acha que mais representa a sua dor',
 'de  a  qual o nivel de intensidade da sua dor atualmente',
 'qual a intensidade da sua dor']

#### **Análise Semântica Latente (LSA)**

---

Análise Semântica Latente (LSA, do inglês Latent Semantic Analysis) é uma técnica de processamento de linguagem natural (NLP) utilizada para analisar relações entre um conjunto de documentos e os termos que eles contêm. A técnica é baseada em matemática linear e estatística, e tem como objetivo descobrir relações semânticas entre palavras e documentos.

Iremos utilizar esse método neste estudo pois lendo nossas frases percebi que de certa forma são usadas palavras diferentes para expressar a mesma coisa. Está aí o motivo pela escolha deste método: **ele é útil para calcular o score de semelhança entre frases que usam palavras diferentes para expressar conceitos semelhantes.**





Vamos começar importando algumas bibliotecas:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer # Vetores
from sklearn.decomposition import TruncatedSVD # Método LSA
from sklearn.metrics.pairwise import cosine_similarity # Calcular a similaridade do cosseno entre dois vetores

# Processamento de linguagem natural
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') # Baixando nossas stopwords (palavras que podem ser consideradas irrelevantes)

Feito nossas importações, bora continuar a aplicação do nosso método:

*Matriz TF-IDF*

---

Vamos criar uma matriz TF-IDF a partir dos textos pré-processados, 
que é uma representação numérica dos textos que pode ser usada para calcular a similaridade entre eles usando técnicas como LSA:

In [8]:
# Salvando palavras em português que podem ser consideradas irrelevantes na variável 'stopwords'

stop_words = stopwords.words('portuguese')

In [9]:
# Matriz tf-idf

vectorizer = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(texto_limpo)

*Método LSA*

---

In [10]:
# Instanciando nosso modelo LSA
lsa_model = TruncatedSVD(n_components=2)

# Aplicando nosso modelo na matriz TF-IDF
lsa_matrix = lsa_model.fit_transform(tfidf_matrix)

*Resultados*

---

Aplicado nosso método, agora vamos avaliar calculando a similaridade cosseno entre nossa frase original e comparativas.

A similaridade cosseno é uma medida que indica o grau de similaridade entre dois vetores em um espaço vetorial. No caso de documentos, podemos pensar que cada documento é um vetor que representa a frequência de cada termo do vocabulário em um espaço vetorial:

In [11]:
# Cálculo similaridade cosseno
cos_similarities = cosine_similarity(lsa_matrix[0].reshape(1, -1), lsa_matrix[1:])

Bora plotar nossos resultados em um DataFrame:

In [12]:
# Importando biblioteca do Pandas
import pandas as pd

In [13]:
# Loop for: Adicionando frases e score de semelhança em uma lista vazia "similarities_list"

similarities_list = []
for i, cos_sim in enumerate(cos_similarities[0]):
    similarities_list.append((documents[0], documents[i+1], cos_sim))

In [14]:
# Criando nosso DataFrame
df = pd.DataFrame(similarities_list, columns=['Original', 'Comparativas', 'Score de Semelhança'])

In [15]:
# Visualização do DataFrame
df

,Original,Comparativas,Score de Semelhança
0,"Olhando para a escala na parede, qual valor in...","De acordo com a escala de dor ali na parede, q...",0.984824
1,"Olhando para a escala na parede, qual valor in...","De 0 a 10, qual o nível de intensidade da sua ...",0.013619
2,"Olhando para a escala na parede, qual valor in...",Qual a intensidade da sua dor?,0.163794


Esses são os resultados que obtivemos:

**Frase original:**
- *Olhando para a escala na parede, qual valor indicaria melhor a sua dor hoje?*

**Frases comparativas:**
- *De acordo com a escala de dor ali na parede, qual valor você acha que mais representa a sua dor?*
  - Apresentou um score de similaridade = 98,48%

- *De 0 a 10, qual o nível de intensidade da sua dor atualmente?*

  - Apresentou um score de similaridade = 1,36%

- *Qual a intensidade da sua dor?*

  - Apresentou um score de similaridade = 16,37%



<img src="https://media.giphy.com/media/L2xcBudpN3nXTfV8ya/giphy.gif" alt="win" width="600"/>

#### **Conclusão**

---

Através deste estudo, quis trazer ao conhecimento como implementar medidas de similaridade entre textos, a fim de deixar o entendimento mais transparente e direto. Algo que também está no escopo deste estudo foi o processo de tratamento e limpeza de texto e por fim a implementação da análise semântica (LSA).

Obviamente, como todos os estudos, este não é uma exceção e poderá ser refinado, assim como estará aberto para novas ideias, feedbacks e ferramentas a serem implementadas.

Um dos principais resultados que obtivemos aqui foi a aplicação de análise semântica latente. **Se pensarmos como material de 'estudos' conseguimos criar um bom modelo** apresentando mais de 90% de similaridade em uma das 3 frases comparativas. Claramente podemos melhorar o resultado nas outras duas frases, porém é um fato que o algoritmo não vai capturar sempre todas as nuances semânticas da linguagem natural. Outro ponto importante que levaria a resultados melhores seria trabalharmos com mais dados e testar outros algoritmos com técnicas de GridSearch para alavancar os hiperparâmetros por exemplo (o que acabou fugindo do escopo deste estudo).
